In [18]:
# carregando dados 
import pandas as pd
import numpy as np

df_maio = pd.read_csv('dados/PNAD_COVID_052020.csv')
df_junho = pd.read_csv('dados/PNAD_COVID_062020.csv')
df_julho = pd.read_csv('dados/PNAD_COVID_072020.csv')

**Questões:**

1.	Qual é o percentual de pessoas que, ao apresentar ao menos 3 sintomas, não procurou nenhum serviço de saúde?

2.	Entre as pessoas que procuraram atendimento de saúde, qual foi o tipo de serviço mais utilizado (público vs. privado)?

3.	Dentre as pessoas que não buscaram atendimento de saúde, qual porcentagem fez uso de auto medicação?

4.	Quais providências foram mais adotadas para tratar sintomas (ex.: ficar em casa vs. automedicação) em pacientes com 3 ou mais sintomas?

5.	Qual a porcentagem de pessoas hospitalizadas que necessitou de ventilação mecânica?

6.  Qual a proporção de pessoas com ao menos 3 sintomas e com plano de saúde que optaram por utilizar o SUS?

7.  Entre as pessoas que buscaram atendimento, qual foi a taxa de internação hospitalar?




Questão 1: Qual é o percentual de pessoas que, ao apresentar ao menos 3 sintomas, não procurou nenhum serviço de saúde?

| Mês   | Procurou       | Não procurou      |
|-------|-----------|-----------|
| Maio  | 27.676831 | 72.323169 |
| Junho | 35.899311 | 64.100689 |
| Julho | 40.235186 | 59.764814 |



In [10]:
# Questão 1

# Selecionando colunas relevantes para a resolução
df_q1_maio = df_maio[["V1013","B0011","B0012","B0013","B0014","B0015","B0016","B0017","B0018","B0019", "B00110", "B00111", "B00112", "B002"]]
df_q1_junho = df_junho[["V1013","B0011","B0012","B0013","B0014","B0015","B0016","B0017","B0018","B0019", "B00110", "B00111", "B00112", "B002"]]
df_q1_julho = df_julho[["V1013","B0011","B0012","B0013","B0014","B0015","B0016","B0017","B0018","B0019", "B00110", "B00111", "B00112", "B002"]]

# Unir os três DataFrames em um só
df_q1_total = pd.concat([df_q1_maio, df_q1_junho, df_q1_julho], axis=0)

# Resetar o índice para organizar as linhas
df_q1_total.reset_index(drop=True, inplace=True)

# Remover linhas onde B002 é igual a 9 ou NaN
df_q1_total = df_q1_total[(df_q1_total["B002"] != 9) & (df_q1_total["B002"].notna())]

# Filtrar linhas onde o valor 1 ocorre ao menos 3 vezes nas colunas de índice 1 a 12
df_q1_total = df_q1_total[df_q1_total.iloc[:, 1:13].eq(1).sum(axis=1) >= 3]

# Agrupar por V1013 (mês) e calcular a proporção dos valores 1 (procurou serviço de saúde) e 2 (não procurou serviço de saúde) em B002
percentual_por_mes = df_q1_total.groupby("V1013")["B002"].value_counts(normalize=True).unstack().fillna(0) * 100

# Exibir apenas as porcentagens de interesse (1 e 2)
percentual_interesse = percentual_por_mes[[1, 2]]

# Renomear o índice para representar os meses
percentual_interesse.index = percentual_interesse.index.map({5: "Maio", 6: "Junho", 7: "Julho"})

# Salvando df_q1_total em xlsx para futura conferencia
df_q1_total.to_excel("dados/clean_data/q1.xlsx")

# Exibir as porcentagens de 1 e 2 para cada mês
print(percentual_interesse)


B002         1.0        2.0
V1013                      
Maio   27.676831  72.323169
Junho  35.899311  64.100689
Julho  40.235186  59.764814


Questão 2: Entre as pessoas que procuraram atendimento de saúde, qual foi o tipo de serviço mais utilizado (público vs. privado)?

| Mês   | Apenas público | Apenas privado | Ambos (público e privado) | Não especificado |
|-------|---------------|----------------------------------|---------------|--------------------------------|
| Maio  | 75.066313    | 19.330239                      | 2.718833     | 2.884615                  |
| Junho | 78.215223    | 17.340332                      | 2.502187     | 1.942257                     |
| Julho | 76.653359     | 18.347509                     | 2.638431     | 2.360701                       |





In [35]:
# Questão 2

# Selecionando colunas relevantes para a resolução
df_q2_maio = df_maio[["V1013","B002","B0041","B0042","B0043","B0044","B0045","B0046"]]
df_q2_junho = df_junho[["V1013","B002","B0041","B0042","B0043","B0044","B0045","B0046"]]
df_q2_julho = df_julho[["V1013","B002","B0041","B0042","B0043","B0044","B0045","B0046"]]

# Unir os três DataFrames em um só
df_q2_total = pd.concat([df_q2_maio, df_q2_junho, df_q2_julho], axis=0)

# Selecionar apenas linhas onde a pessoa procurou estabelecimento de saúde
df_q2_total = df_q2_total[(df_q2_total["B002"] != 2) & (df_q2_total["B002"] != 9) & (df_q2_total["B002"].notna())]

# Criar um dicionário para armazenar os resultados por mês
resultados = {}

# Agrupar por V1013 (mês) e iterar pelos grupos de meses (5 - Maio, 6 - Junho, 7 - Julho)
for mes, grupo in df_q2_total.groupby("V1013"):
    # Verificar o uso de serviços de saúde pública e privada
    usou_publico = grupo[["B0041", "B0042", "B0043"]].eq(1).any(axis=1)
    usou_privado = grupo[["B0044", "B0045", "B0046"]].eq(1).any(axis=1)

    # Identificar os diferentes grupos de uso
    apenas_publico = usou_publico & ~usou_privado
    apenas_privado = ~usou_publico & usou_privado
    ambos = usou_publico & usou_privado
    nao_especificado = ~usou_publico & ~usou_privado  # Casos onde a pessoa não usou nem serviços públicos nem privados

    # Calcular as proporções para cada categoria
    percentual_apenas_publico = apenas_publico.mean() * 100
    percentual_apenas_privado = apenas_privado.mean() * 100
    percentual_ambos = ambos.mean() * 100
    percentual_nao_especificado = nao_especificado.mean() * 100

    # Armazenar os resultados no dicionário com o nome do mês correspondente
    resultados[mes] = {
        "Porcentagem apenas público": percentual_apenas_publico,
        "Porcentagem apenas privado": percentual_apenas_privado,
        "Porcentagem ambos (público e privado)": percentual_ambos,
        "Porcentagem não especificado": percentual_nao_especificado
    }

# Converter o dicionário para um DataFrame para exibir o resultado
resultados_df = pd.DataFrame(resultados).T

# Renomear o índice para representar os meses
resultados_df.index = resultados_df.index.map({5: "Maio", 6: "Junho", 7: "Julho"})

# Salvando df_q2_total em xlsx para futura conferência
df_q2_total.to_excel("dados/clean_data/q2.xlsx")

# Exibir o resultado
print(resultados_df)


       Porcentagem apenas público  Porcentagem apenas privado  \
Maio                    75.066313                   19.330239   
Junho                   78.215223                   17.340332   
Julho                   76.653359                   18.347509   

       Porcentagem ambos (público e privado)  Porcentagem não especificado  
Maio                                2.718833                      2.884615  
Junho                               2.502187                      1.942257  
Julho                               2.638431                      2.360701  


Questão 3: Dentre as pessoas que não buscaram atendimento de saúde, qual porcentagem fez uso de auto medicação?

| Mês   | Porcentagem de auto medicação | 
|-------|-----------|
| Maio  | 57.202902|
| Junho | 59.418362 |
| Julho | 59.579879 |





In [12]:
# Questão 3

# Selecionando colunas relevantes para a resolução
df_q3_maio = df_maio[["V1013","B002", "B0033"]]
df_q3_junho = df_junho[["V1013","B002", "B0033"]]
df_q3_julho = df_julho[["V1013","B002", "B0033"]]

# Unir os três DataFrames em um só
df_q3_total = pd.concat([df_q3_maio, df_q3_junho, df_q3_julho], axis=0)

# Seleciondo apenas linhas onde a pessoa não procurou estabelecimento de saude
df_q3_total = df_q3_total[(df_q3_total["B002"] != 1) & (df_q3_total["B002"] != 9) & (df_q3_total["B002"].notna())]

# Agrupar por V1013 (mês) e calcular a porcentagem de ocorrências de 1 em B0033
percentuais = df_q3_total.groupby("V1013")[["B0033"]].apply(lambda x: (x == 1).mean() * 100)

# Renomear o índice para representar os meses
percentuais.index = percentuais.index.map({5: "Maio", 6: "Junho", 7: "Julho"})

# Salvando df_q1_total em xlsx para futura conferencia
df_q3_total.to_excel("dados/clean_data/q3.xlsx")

# Exibir o resultado
print(percentuais)

           B0033
V1013           
Maio   57.202902
Junho  59.418362
Julho  59.579879


Questão 4: Quais providências foram mais adotadas para tratar sintomas (ex.: ficar em casa vs. automedicação) em pacientes com 3 ou mais sintomas?

| Mês   | Ficar em casa | Ligar para profissional de saude | Automedicação | Medicação c/ orientação médica | Visita profissional do SUS | Visita profissional rede particular | Outros    |
|-------|---------------|----------------------------------|---------------|--------------------------------|----------------------------|-------------------------------------|-----------|
| Maio  | 63.917044     | 8.194293                       | 48.627787     | 13.262124                    | 3.181039                 | 1.130516                           | 2.993919 |
| Junho | 56.203008     | 7.205514                       | 43.504595     | 11.549708                      | 3.080618                   | 0.845865                           | 2.255639 |
| Julho | 50.817972     | 4.873272                       | 41.244240      | 10.288018                      | 3.260369                  | 0.599078                            | 1.993088 |

In [25]:
# Questão 4

# Selecionando colunas relevantes para a resolução
df_q4_maio = df_maio[["V1013","B0011","B0012","B0013","B0014","B0015","B0016","B0017","B0018","B0019", "B00110", "B00111", "B00112", "B0031", "B0032", "B0033", "B0034", "B0035", "B0036", "B0037"]]
df_q4_junho = df_junho[["V1013","B0011","B0012","B0013","B0014","B0015","B0016","B0017","B0018","B0019", "B00110", "B00111", "B00112", "B0031", "B0032", "B0033", "B0034", "B0035", "B0036", "B0037"]]
df_q4_julho = df_julho[["V1013","B0011","B0012","B0013","B0014","B0015","B0016","B0017","B0018","B0019", "B00110", "B00111", "B00112", "B0031", "B0032", "B0033", "B0034", "B0035", "B0036", "B0037"]]

# Unir os três DataFrames em um só
df_q4_total = pd.concat([df_q4_maio, df_q4_junho, df_q4_julho], axis=0)

# Remover linhas com valores NA nas colunas de providencias tomadas ("B0031" até "B0037")
df_q4_total_limpo = df_q4_total.dropna(subset=["B0031", "B0032", "B0033", "B0034", "B0035", "B0036", "B0037"])

# Filtrar linhas onde o valor 1 ocorre ao menos 3 vezes nas colunas de índice 1 a 13 (apenas pacientes com 3 ou mais sintomas)
df_q4_total = df_q4_total[df_q4_total.iloc[:, 1:13].eq(1).sum(axis=1) >= 3]

# Agrupar por V1013 (mês) e calcular a porcentagem de ocorrências de 1 em B0031 até B0036
percentuais_q4 = df_q4_total.groupby("V1013")[["B0031", "B0032", "B0033", "B0034", "B0035", "B0036", "B0037"]].apply(lambda x: (x == 1).mean() * 100)

# Renomear o índice para representar os meses
percentuais_q4.index = percentuais_q4.index.map({5: "Maio", 6: "Junho", 7: "Julho"})

# Salvando df_q1_total em xlsx para futura conferencia
df_q4_total.to_excel("dados/clean_data/q4.xlsx")

# Exibir o resultado
print(percentuais_q4)



           B0031     B0032      B0033      B0034     B0035     B0036     B0037
V1013                                                                         
Maio   63.917044  8.194293  48.627787  13.262124  3.181039  1.130516  2.993919
Junho  56.203008  7.205514  43.504595  11.549708  3.080618  0.845865  2.255639
Julho  50.817972  4.873272  41.244240  10.288018  3.260369  0.599078  1.993088


Questão 5: Qual a porcentagem de pessoas hospitalizadas que necessitou de ventilação mecânica?

| Mês   | Porcentagem de ventilação mecânica | 
|-------|-----------|
| Maio  | 25.000000 |
| Junho | 27.312775 |
| Julho | 25.660377 |

In [49]:
# Questão 5

# Selecionando colunas relevantes para a resolução
df_q5_maio = df_maio[["V1013","B002", "B005", "B006"]]
df_q5_junho = df_junho[["V1013","B002", "B005", "B006"]]
df_q5_julho = df_julho[["V1013","B002", "B005", "B006"]]

# Unir os três DataFrames em um só
df_q5_total = pd.concat([df_q5_maio, df_q5_junho, df_q5_julho], axis=0)

# Selecionando apenas linhas onde a pessoa procurou estabelecimento de saúde
df_q5_total = df_q5_total[(df_q5_total["B002"] != 2) & (df_q5_total["B002"] != 9) & (df_q5_total["B002"].notna())]

# Selecionando apenas linhas onde a pessoa foi hospitalizada
df_q5_total = df_q5_total[(df_q5_total["B005"] != 2) & (df_q5_total["B005"] != 3) & (df_q5_total["B005"] != 9) & (df_q5_total["B005"].notna())]

# Agrupar por V1013 (mês) e calcular a porcentagem de ocorrências de 1 em B006
percentuais = df_q5_total.groupby("V1013")[["B006"]].apply(lambda x: (x == 1).mean() * 100)

# Renomear o índice para representar os meses
percentuais.index = percentuais.index.map({5: "Maio", 6: "Junho", 7: "Julho"})

# Salvando df_q1_total em xlsx para futura conferencia
df_q5_total.to_excel("dados/clean_data/q5.xlsx")

# Exibir o resultado
print(percentuais)


            B006
V1013           
Maio   25.000000
Junho  27.312775
Julho  25.660377


Questão 6: Qual a proporção de pessoas com ao menos 3 sintomas e com plano de saúde que optaram por utilizar o SUS?

| Mês   | Apenas público | Apenas privado | Ambos (público e privado) | Não especificado |
|-------|---------------|----------------------------------|---------------|--------------------------------|
| Maio  | 38.032787     | 53.442623                      | 5.464481     | 3.060109                   |
| Junho | 37.377451     | 52.573529                       | 7.720588      | 2.328431                     |
| Julho | 33.794694     | 56.978085                      | 5.190311      | 4.036909                       |

In [34]:
# Questão 6

# Selecionando colunas relevantes para a resolução
df_q6_maio = df_maio[["V1013","B0011","B0012","B0013","B0014","B0015","B0016","B0017","B0018","B0019", "B00110", "B00111", "B00112","B007","B0041","B0042","B0043","B0044", "B0045", "B0046","B002"]]
df_q6_junho = df_junho[["V1013","B0011","B0012","B0013","B0014","B0015","B0016","B0017","B0018","B0019", "B00110", "B00111", "B00112","B007","B0041","B0042","B0043","B0044", "B0045", "B0046","B002"]]
df_q6_julho = df_julho[["V1013","B0011","B0012","B0013","B0014","B0015","B0016","B0017","B0018","B0019", "B00110", "B00111", "B00112","B007","B0041","B0042","B0043","B0044", "B0045", "B0046","B002"]]

# Unir os três DataFrames em um só
df_q6_total = pd.concat([df_q6_maio, df_q6_junho, df_q6_julho], axis=0)

# Selecionando apenas linhas onde a pessoa procurou estabelecimento de saúde
df_q6_total = df_q6_total[(df_q6_total["B002"] != 2) & (df_q6_total["B002"] != 9) & (df_q6_total["B002"].notna())]

# Filtrar linhas onde o valor 1 ocorre ao menos 3 vezes nas colunas de índice 1 a 13 (apenas pacientes com 3 ou mais sintomas)
df_q6_total = df_q6_total[df_q6_total.iloc[:, 1:13].eq(1).sum(axis=1) >= 3]

# Selecionando apenas pessoas com plano de saúde
df_q6_total = df_q6_total[(df_q6_total["B007"] != 2) & (df_q6_total["B007"] != 9)]

# Criar um dicionário para armazenar os resultados por mês
resultados = {}

# Agrupar por V1013 (mês) e iterar pelos grupos de meses (5 - Maio, 6 - Junho, 7 - Julho)
for mes, grupo in df_q6_total.groupby("V1013"):
    # Verificar o uso de serviços de saúde pública e privada
    usou_publico = grupo[["B0041", "B0042", "B0043"]].eq(1).any(axis=1)
    usou_privado = grupo[["B0044", "B0045", "B0046"]].eq(1).any(axis=1)

    # Identificar os diferentes grupos de uso
    apenas_publico = usou_publico & ~usou_privado
    apenas_privado = ~usou_publico & usou_privado
    ambos = usou_publico & usou_privado
    nao_especificado = ~usou_publico & ~usou_privado  # Casos onde a pessoa não usou nem serviços públicos nem privados

    # Calcular as proporções para cada categoria
    percentual_apenas_publico = apenas_publico.mean() * 100
    percentual_apenas_privado = apenas_privado.mean() * 100
    percentual_ambos = ambos.mean() * 100
    percentual_nao_especificado = nao_especificado.mean() * 100

    # Armazenar os resultados no dicionário com o nome do mês correspondente
    resultados[mes] = {
        "Porcentagem apenas público": percentual_apenas_publico,
        "Porcentagem apenas privado": percentual_apenas_privado,
        "Porcentagem ambos (público e privado)": percentual_ambos,
        "Porcentagem não especificado": percentual_nao_especificado
    }

# Converter o dicionário para um DataFrame para exibir o resultado
resultados_df = pd.DataFrame(resultados).T

# Renomear o índice para representar os meses
resultados_df.index = resultados_df.index.map({5: "Maio", 6: "Junho", 7: "Julho"})

# Salvando df_q6_total em xlsx para futura conferência
df_q6_total.to_excel("dados/clean_data/q6.xlsx")

# Exibir o resultado
print(resultados_df)


       Porcentagem apenas público  Porcentagem apenas privado  \
Maio                    38.032787                   53.442623   
Junho                   37.377451                   52.573529   
Julho                   33.794694                   56.978085   

       Porcentagem ambos (público e privado)  Porcentagem não especificado  
Maio                                5.464481                      3.060109  
Junho                               7.720588                      2.328431  
Julho                               5.190311                      4.036909  


Questão 7: Entre as pessoas que buscaram atendimento, qual foi a taxa de internação hospitalar?

| Mês   | Porcentagem de internaçào hospitalar | 
|-------|-----------|
| Maio  | 3.315650|
| Junho | 3.972003 |
| Julho | 4.599896 |

In [36]:
# Questão 7

# Selecionando colunas relevantes para a resolução
df_q7_maio = df_maio[["V1013","B002", "B005"]]
df_q7_junho = df_junho[["V1013","B002", "B005"]]
df_q7_julho = df_julho[["V1013","B002", "B005"]]

# Unir os três DataFrames em um só
df_q7_total = pd.concat([df_q7_maio, df_q7_junho, df_q7_julho], axis=0)

# Selecionando apenas linhas onde a pessoa procurou estabelecimento de saúde
df_q7_total = df_q7_total[(df_q7_total["B002"] != 2) & (df_q7_total["B002"] != 9) & (df_q7_total["B002"].notna())]

# Agrupar por V1013 (mês) e calcular a porcentagem de ocorrências de 1 em B005
percentuais = df_q7_total.groupby("V1013")[["B005"]].apply(lambda x: (x == 1).mean() * 100)

# Renomear o índice para representar os meses
percentuais.index = percentuais.index.map({5: "Maio", 6: "Junho", 7: "Julho"})

# Salvando df_q1_total em xlsx para futura conferencia
df_q7_total.to_excel("dados/clean_data/q7.xlsx")

# Exibir o resultado
print(percentuais)

           B005
V1013          
Maio   3.315650
Junho  3.972003
Julho  4.599896
